In [ ]:
import datetime
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
%matplotlib inline



In [ ]:
%load_ext autoreload
%autoreload 2
import c14
import c14.models.liver_new_mitosis as lm


In [ ]:
data = pd.read_csv('C14data_liver_samples_20211020.csv')
data = data.groupby(['type', 'sample', 'ploidy', 'pathology']).mean().dropna(how='all').reset_index()
#data['Dbirth'] -= 0.5
data['age'] = data['Dcoll'] - data['Dbirth']
data = data.query('type == "hepatocyte" and pathology != ["Y", "C"]' )

m2n = data['ploidy']=="2n"
m4n = data['ploidy']=="4n"
m2n4n = data['ploidy']=="2n4n"
mC2n = data['ploidy']=="C2n"
mC4n = data['ploidy']=="C4n"

with pd.HDFStore('point_est.pandas') as st:
    point_est = st['F_IS_spline_newploidy'] 
    
    
paraP =dict(point_est.loc['POP1spline','median'].dropna())
mP = c14.models.liver_new_mitosis.POP1(dnatotal=lm.dnatotal_spline)
mP.set_parameters_phy(paraP,mode='bayes')
opP=c14.optimize(mP,data=c14.exp_data(data),step_size=0.1)
simdataP=opP.calc_sim_data_dict(\
    mP.transform_physical_to_fit(paraP,mode='bayes'),mode='bayes')    
    
paraRm =dict(point_est.loc['Rmspline','median'].dropna())
mRm = c14.models.liver_new_mitosis.Rm(dnatotal=lm.dnatotal_spline)
mRm.set_parameters_phy(paraRm,mode='bayes')
opRm=c14.optimize(mRm,data=c14.exp_data(data),step_size=0.1)
simdataRm=opRm.calc_sim_data_dict(\
    mRm.transform_physical_to_fit(paraRm,mode='bayes'),mode='bayes')

paraPOP3p_2x2n =dict(point_est.loc['POP3p_2x2n_NPspline','median'].dropna())
mPOP3p_2x2n = c14.models.liver_new_mitosis.POP3p_2x2n_NP(dnatotal=lm.dnatotal_spline)
mPOP3p_2x2n.set_parameters_phy(paraPOP3p_2x2n,mode='bayes')
opPOP3p_2x2n=c14.optimize(mPOP3p_2x2n ,data=c14.exp_data(data),step_size=0.1)
simdataPOP3p_2x2n=opPOP3p_2x2n.calc_sim_data_dict(\
    mPOP3p_2x2n.transform_physical_to_fit(paraPOP3p_2x2n,mode='bayes'),mode='bayes')


In [ ]:
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.lines import Line2D
from matplotlib import colors
plt.figure(11,figsize=[6.4*3/1.5, 2*4.8/1.5])
sh=(2,3)
ax1 = plt.subplot2grid(sh,(0,0))
ax2 = plt.subplot2grid(sh,(0,1))
ax3 = plt.subplot2grid(sh,(0,2))
ax4 = plt.subplot2grid(sh,(1,0))
ax5 = plt.subplot2grid(sh,(1,1))
plt.subplot2grid(sh,(1,2)).remove()



cmap = colors.ListedColormap(['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown'])
bounds=[2004,2012,2016,2017,2018,2019,2020]
norm = colors.BoundaryNorm(bounds, cmap.N)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

#divider = make_axes_locatable(ax3)
#cax = divider.append_axes(position='right', size='2%', pad=0.1)
#plt.colorbar(sm, cax=cax)
#cax.set_ylabel('Collection date')
marker1='o'
marker2='x'
size1=40
size2=30

ax1.set_title('2n')
datam = data[m2n]
simd = simdataP[m2n]
ax=ax1
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)

ax2.set_title('4n')   
datam = data[m4n]
simd = simdataP[m4n]
ax=ax2
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)

ax3.set_title('2n4n')    
datam = data[m2n4n]
simd = simdataP[m2n4n]
ax=ax3
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)
    
ax4.set_title('C2n')   
datam = data[mC2n]
simd = simdataP[mC2n]
ax=ax4
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)

ax5.set_title('C4n')    
datam = data[mC4n]
simd = simdataP[mC4n]
ax=ax5
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)

    

lims = np.array([ax1.get_ylim(),ax2.get_ylim(),ax3.get_ylim(),(-0.05,0.3)])
yl = (lims.min(),lims.max())

a = np.linspace(1930,2020,1000)
ax1.plot(a,c14.Catm().lin(a),':',color='gray')
ax2.plot(a,c14.Catm().lin(a),':',color='gray')
ax3.plot(a,c14.Catm().lin(a),':',color='gray')
ax4.plot(a,c14.Catm().lin(a),':',color='gray')
ax5.plot(a,c14.Catm().lin(a),':',color='gray')
#yl = (m.Catm.lin(a).min(),m.Catm.lin(a).max()*1.01)

ax4.set_xlabel(r'Birth date')
ax5.set_xlabel(r'Birth date')
ax3.set_xlabel(r'Birth date')
ax1.set_ylabel(r'$\Delta14C$')
ax4.set_ylabel(r'$\Delta14C$')

ax2.set_ylabel('')
ax3.set_ylabel('')
ax5.set_ylabel('')
ax2.set_yticklabels('')
ax3.set_yticklabels('')
ax5.set_yticklabels('')



plt.tight_layout(w_pad=0)
plt.subplots_adjust(wspace=0.018)
ax1.set_ylim(yl)
ax2.set_ylim(yl)
ax3.set_ylim(yl)
ax4.set_ylim(yl)
ax5.set_ylim(yl)


ax1.set_xlim(data.Dbirth.min()-5,2020)#data.Dbirth.max()+5+15)
ax2.set_xlim(ax1.get_xlim())
ax3.set_xlim(ax1.get_xlim())
ax4.set_xlim(ax1.get_xlim())
ax5.set_xlim(ax1.get_xlim())
custom_lines = [Line2D([0], [0], color='black',marker=marker1,lw=0 ),
                Line2D([0], [0], color='black',marker=marker2,lw=0 ),
                Line2D([0], [0], color='black',lw=0,alpha=1 ),
                Line2D([0], [0], color=sm.to_rgba(2004),marker='s', lw=0),
                Line2D([0], [0], color=sm.to_rgba(2012),marker='s', lw=0)]+\
                [Line2D([0], [0], color=sm.to_rgba(y),marker='s', lw=0) for y in range(2016,2020)]
l=ax5.legend(custom_lines, ['Data','Model','Collection year','2004', '2012']+list(range(2016,2020)),\
          bbox_to_anchor=(1.1, 0.9),frameon=True,title='')
t =l.get_texts()[2]
t.set_position((-28,0))
#plt.tight_layout()
plt.savefig('model_data_comp_POP1_newploidy.pdf', bbox_inches='tight')
plt.savefig('model_data_comp_POP1_newploidy.png', dpi=600,bbox_inches='tight')


In [ ]:
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.lines import Line2D
from matplotlib import colors
plt.figure(11,figsize=[6.4*3/1.5, 2*4.8/1.5])
sh=(2,3)
ax1 = plt.subplot2grid(sh,(0,0))
ax2 = plt.subplot2grid(sh,(0,1))
ax3 = plt.subplot2grid(sh,(0,2))
ax4 = plt.subplot2grid(sh,(1,0))
ax5 = plt.subplot2grid(sh,(1,1))
plt.subplot2grid(sh,(1,2)).remove()



cmap = colors.ListedColormap(['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown'])
bounds=[2004,2012,2016,2017,2018,2019,2020]
norm = colors.BoundaryNorm(bounds, cmap.N)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

#divider = make_axes_locatable(ax3)
#cax = divider.append_axes(position='right', size='2%', pad=0.1)
#plt.colorbar(sm, cax=cax)
#cax.set_ylabel('Collection date')
marker1='o'
marker2='x'
size1=40
size2=30

ax1.set_title('2n')
datam = data[m2n]
simd = simdataRm[m2n]
ax=ax1
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)

ax2.set_title('4n')   
datam = data[m4n]
simd = simdataRm[m4n]
ax=ax2
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)

ax3.set_title('2n4n')    
datam = data[m2n4n]
simd = simdataRm[m2n4n]
ax=ax3
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)
    
ax4.set_title('C2n')   
datam = data[mC2n]
simd = simdataRm[mC2n]
ax=ax4
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)

ax5.set_title('C4n')    
datam = data[mC4n]
simd = simdataRm[mC4n]
ax=ax5
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)

    

lims = np.array([ax1.get_ylim(),ax2.get_ylim(),ax3.get_ylim(),(-0.05,0.3)])
yl = (lims.min(),lims.max())

a = np.linspace(1930,2020,1000)
ax1.plot(a,c14.Catm().lin(a),':',color='gray')
ax2.plot(a,c14.Catm().lin(a),':',color='gray')
ax3.plot(a,c14.Catm().lin(a),':',color='gray')
ax4.plot(a,c14.Catm().lin(a),':',color='gray')
ax5.plot(a,c14.Catm().lin(a),':',color='gray')
#yl = (m.Catm.lin(a).min(),m.Catm.lin(a).max()*1.01)

ax4.set_xlabel(r'Birth date')
ax5.set_xlabel(r'Birth date')
ax3.set_xlabel(r'Birth date')
ax1.set_ylabel(r'$\Delta14C$')
ax4.set_ylabel(r'$\Delta14C$')

ax2.set_ylabel('')
ax3.set_ylabel('')
ax5.set_ylabel('')
ax2.set_yticklabels('')
ax3.set_yticklabels('')
ax5.set_yticklabels('')



plt.tight_layout(w_pad=0)
plt.subplots_adjust(wspace=0.018)
ax1.set_ylim(yl)
ax2.set_ylim(yl)
ax3.set_ylim(yl)
ax4.set_ylim(yl)
ax5.set_ylim(yl)


ax1.set_xlim(data.Dbirth.min()-5,2020)#data.Dbirth.max()+5+15)
ax2.set_xlim(ax1.get_xlim())
ax3.set_xlim(ax1.get_xlim())
ax4.set_xlim(ax1.get_xlim())
ax5.set_xlim(ax1.get_xlim())
custom_lines = [Line2D([0], [0], color='black',marker=marker1,lw=0 ),
                Line2D([0], [0], color='black',marker=marker2,lw=0 ),
                Line2D([0], [0], color='black',lw=0,alpha=1 ),
                Line2D([0], [0], color=sm.to_rgba(2004),marker='s', lw=0),
                Line2D([0], [0], color=sm.to_rgba(2012),marker='s', lw=0)]+\
                [Line2D([0], [0], color=sm.to_rgba(y),marker='s', lw=0) for y in range(2016,2020)]
l=ax5.legend(custom_lines, ['Data','Model','Collection year','2004', '2012']+list(range(2016,2020)),\
          bbox_to_anchor=(1.1, 0.9),frameon=True,title='')
t =l.get_texts()[2]
t.set_position((-28,0))
#plt.tight_layout()
plt.savefig('model_data_comp_Rm_newploidy.pdf', bbox_inches='tight')
plt.savefig('model_data_comp_Rm_newploidy.png', dpi=600,bbox_inches='tight')


In [ ]:
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.lines import Line2D
from matplotlib import colors
plt.figure(11,figsize=[6.4*3/1.5, 2*4.8/1.5])
sh=(2,3)
ax1 = plt.subplot2grid(sh,(0,0))
ax2 = plt.subplot2grid(sh,(0,1))
ax3 = plt.subplot2grid(sh,(0,2))
ax4 = plt.subplot2grid(sh,(1,0))
ax5 = plt.subplot2grid(sh,(1,1))
plt.subplot2grid(sh,(1,2)).remove()



cmap = colors.ListedColormap(['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown'])
bounds=[2004,2012,2016,2017,2018,2019,2020]
norm = colors.BoundaryNorm(bounds, cmap.N)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

#divider = make_axes_locatable(ax3)
#cax = divider.append_axes(position='right', size='2%', pad=0.1)
#plt.colorbar(sm, cax=cax)
#cax.set_ylabel('Collection date')
marker1='o'
marker2='x'
size1=40
size2=30

ax1.set_title('2n')
datam = data[m2n]
simd = simdataPOP3p_2x2n[m2n]
ax=ax1
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)

ax2.set_title('4n')   
datam = data[m4n]
simd = simdataPOP3p_2x2n[m4n]
ax=ax2
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)

ax3.set_title('2n4n')    
datam = data[m2n4n]
simd = simdataPOP3p_2x2n[m2n4n]
ax=ax3
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)
    
ax4.set_title('C2n')   
datam = data[mC2n]
simd = simdataPOP3p_2x2n[mC2n]
ax=ax4
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)

ax5.set_title('C4n')    
datam = data[mC4n]
simd = simdataPOP3p_2x2n[mC4n]
ax=ax5
ax.scatter(datam.Dbirth, datam.d14C, c=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker1,zorder=3,s=size1)
ax.scatter(datam.Dbirth,simd , color=sm.to_rgba(datam.Dcoll), facecolor=sm.to_rgba(datam.Dcoll),
             marker=marker2,zorder=2,s=size2)
for index,row in datam.iterrows():
    ax.plot([row.Dbirth,row.Dbirth],[row.d14C,simd[index]],':', color=sm.to_rgba(row.Dcoll),alpha=1,zorder=1)

    

lims = np.array([ax1.get_ylim(),ax2.get_ylim(),ax3.get_ylim(),(-0.05,0.3)])
yl = (lims.min(),lims.max())

a = np.linspace(1930,2020,1000)
ax1.plot(a,c14.Catm().lin(a),':',color='gray')
ax2.plot(a,c14.Catm().lin(a),':',color='gray')
ax3.plot(a,c14.Catm().lin(a),':',color='gray')
ax4.plot(a,c14.Catm().lin(a),':',color='gray')
ax5.plot(a,c14.Catm().lin(a),':',color='gray')
#yl = (m.Catm.lin(a).min(),m.Catm.lin(a).max()*1.01)



ax2.set_ylabel('')
ax3.set_ylabel('')
ax5.set_ylabel('')
ax2.set_yticklabels('')
ax3.set_yticklabels('')
ax5.set_yticklabels('')



plt.tight_layout(w_pad=0)
plt.subplots_adjust(wspace=0.018)
ax1.set_ylim(yl)
ax2.set_ylim(yl)
ax3.set_ylim(yl)
ax4.set_ylim(yl)
ax5.set_ylim(yl)

ax4.set_xlabel(r'Birth date')
ax5.set_xlabel(r'Birth date')
ax3.set_xlabel(r'Birth date')
ax1.set_ylabel(r'$\Delta14C$')
ax4.set_ylabel(r'$\Delta14C$')

ax1.set_xlim(data.Dbirth.min()-5,2020)#data.Dbirth.max()+5+15)
ax2.set_xlim(ax1.get_xlim())
ax3.set_xlim(ax1.get_xlim())
ax4.set_xlim(ax1.get_xlim())
ax5.set_xlim(ax1.get_xlim())
custom_lines = [Line2D([0], [0], color='black',marker=marker1,lw=0 ),
                Line2D([0], [0], color='black',marker=marker2,lw=0 ),
                Line2D([0], [0], color='black',lw=0,alpha=1 ),
                Line2D([0], [0], color=sm.to_rgba(2004),marker='s', lw=0),
                Line2D([0], [0], color=sm.to_rgba(2012),marker='s', lw=0)]+\
                [Line2D([0], [0], color=sm.to_rgba(y),marker='s', lw=0) for y in range(2016,2020)]
l=ax5.legend(custom_lines, ['Data','Model','Collection year','2004', '2012']+list(range(2016,2020)),\
          bbox_to_anchor=(1.1, 0.9),frameon=True,title='')
t =l.get_texts()[2]
t.set_position((-28,0))
#plt.tight_layout()
plt.savefig('model_data_comp_POP3p_2x2n_newploidy.pdf', bbox_inches='tight')
plt.savefig('model_data_comp_POP3p_2x2n_newploidy.png', dpi=600,bbox_inches='tight')
